### Facial Recognition Function

In [4]:
import cv2
from cv2 import VideoWriter_fourcc, VideoWriter
import sys
import os
import requests
import base64
import numpy as np

def stream_frame_write(context,frame,shard):
    headers = {
            "Content-Type": "application/json",
            "X-v3io-function": "PutRecords",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }
    data = base64.b64encode(frame)
    payload = {
            "Records": [
                {
                    "Data":  data.decode('utf-8'),
                    "ShardId" : shard
                    }
                ]
          }
    url = "http://v3io-webapi:8081/%s/%s/"% (os.getenv('IGZ_CONTAINER'),os.getenv('TAGGED_VIDEO_STREAM'))
    r = requests.post(url=url, headers=headers,json=payload)   
    return r.text

def handler(context, event):
    shardId = event.shard_id
    event_data=event.body
    
    # Load the cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # To capture video from webcam.
    img = event_data
    img = np.frombuffer(img, dtype=np.uint8)
    img = cv2.imdecode(img, flags=1)
    if bool(os.getenv('ROTATE_180')):
        img = cv2.rotate(img, cv2.ROTATE_180) # For some reason my stream is flipped?
    
    faces = face_cascade.detectMultiScale(img, 1.1, 4)
    
    # Draw the rectangle around each face
    face_count=0
    for (x, y, w, h) in faces:
        face_count += 1
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
    context.logger.info("Face Count: %s"% face_count )
    if face_count > 0 :
        ret, buffer = cv2.imencode('.jpg', img)
        try:
            r = stream_frame_write(context,buffer.tobytes(),shardId)
        except:
            context.logger.error("Failed to save frame for shard %s"% shardId)